### Export values from the Netherlands to other 5 EU countries

In [19]:
import pandas as pd
import os
os.listdir()

# Load the Summarized dataset into a DataFrame for Export value information
export_data = pd.read_excel('data/Summarized Data.xlsx', usecols=['Quarter', 'Price', 'Export Germany', 'Export Belgium', 'Export France', 'Export Italy', 'Export Poland']).dropna()

# Apply border styling
styled_df = export_data.style.set_table_styles(
    [{'selector': 'th', 'props': [('border', '1px solid black')]},
     {'selector': 'td', 'props': [('border', '1px solid black')]}]
)

# Display the styled table
styled_df

,Quarter,Price,Export Germany,Export Belgium,Export France,Export Italy,Export Poland
0,2021 Q1,60.820000,44.117655,21.330007,17.206682,8.232882,6.464029
1,2021 Q2,68.830000,46.797432,22.626315,17.625687,8.438632,6.848701
2,2021 Q3,73.470000,47.022815,22.971691,17.868996,8.190615,6.796111
3,2021 Q4,79.590000,50.446783,24.537921,19.451189,9.228776,7.857002
4,2022 Q1,100.300000,52.094426,25.765264,18.914984,9.436756,7.941483
5,2022 Q2,113.540000,54.533106,27.722722,19.069949,9.441125,7.449511
6,2022 Q3,100.710000,52.237598,25.748810,18.312154,8.402710,7.057325
7,2022 Q4,88.560000,51.860825,26.689289,20.520986,8.964858,7.799943
8,2023 Q1,81.170000,55.508039,26.052519,19.919543,9.985542,8.299103
9,2023 Q2,78.320000,53.140635,25.434880,19.466554,9.420803,7.823181


In [20]:
total_exports = export_data[['Export Germany', 'Export Belgium', 'Export France', 'Export Italy', 'Export Poland']].sum()

sum_exports_df = pd.DataFrame({
    'Country': total_exports.index,
    'Total Export Value from the Netherlands': total_exports.values
})

sum_exports_df = sum_exports_df.sort_values(by='Total Export Value from the Netherlands', ascending=False)

sum_exports_df.reset_index(drop=True, inplace=True)

styled_sum_df = sum_exports_df.style.set_table_styles(
    [{'selector': 'th', 'props': [('border', '1px solid black')]},
     {'selector': 'td', 'props': [('border', '1px solid black')]}]
)

styled_sum_df

,Country,Total Export Value from the Netherlands
0,Export Germany,609.719842
1,Export Belgium,300.845442
2,Export France,226.360506
3,Export Italy,108.017114
4,Export Poland,89.954326


In [21]:
# Visualization of total Export values from the Netherlands to other 5 EU countries

import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create a subplot with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add export data lines for each country
countries = export_data.columns[2:7].tolist()

for i, country in enumerate(countries):
    fig.add_trace(
        go.Scatter(x = export_data['Quarter'], y = export_data[country],
                   mode = 'lines',
                   name = country),
        secondary_y = False,
    )

# Add Price line
fig.add_trace(
    go.Scatter(x = export_data['Quarter'], y = export_data['Price'],
               mode = 'lines',
               name = 'Oil Price'),
    secondary_y = True,
)

# Update layout
fig.update_layout(
    title = 'Export Values by Country and Price Over Time', 
    xaxis = dict(showgrid = True, tickangle = 45),
    yaxis = dict(showgrid = True),
    xaxis_title = 'Year, Quarter',
    yaxis_title = 'Export Values (in Million USD)',
    yaxis2_title = 'Oil Price (in USD)',
)

# Show the figure
fig.show()


In [46]:
# Set 'Quarter' as the index for calculating percentage changes
export_data.set_index('Quarter', inplace=True)

# Calculate quarter-to-quarter percentage change
percent_change = export_data.pct_change() * 100  # This gives the percentage change for each column
percent_change = percent_change.rename(columns=lambda x: f"{x} % Change")

# Drop any NaN values in the first row (since pct_change produces NaN for the first row)
percent_change = percent_change.dropna().reset_index()

# Apply table styling with borders and format only numeric columns to two decimal places
styled_percent_change = percent_change.style.set_table_styles(
    [{'selector': 'th', 'props': [('border', '1px solid black')]},
     {'selector': 'td', 'props': [('border', '1px solid black')]}]
).format({col: "{:.2f}" for col in percent_change.columns if percent_change[col].dtype != "object"})

# Display the styled DataFrame
styled_percent_change


,Quarter,Price % Change,Export Germany % Change,Export Belgium % Change,Export France % Change,Export Italy % Change,Export Poland % Change
0,2021 Q2,13.17,6.07,6.08,2.44,2.50,5.95
1,2021 Q3,6.74,0.48,1.53,1.38,-2.94,-0.77
2,2021 Q4,8.33,7.28,6.82,8.85,12.68,15.61
3,2022 Q1,26.02,3.27,5.00,-2.76,2.25,1.08
4,2022 Q2,13.20,4.68,7.60,0.82,0.05,-6.19
5,2022 Q3,-11.30,-4.21,-7.12,-3.97,-11.00,-5.26
6,2022 Q4,-12.06,-0.72,3.65,12.06,6.69,10.52
7,2023 Q1,-8.34,7.03,-2.39,-2.93,11.39,6.40
8,2023 Q2,-3.51,-4.26,-2.37,-2.27,-5.66,-5.73
9,2023 Q3,10.65,-3.40,2.47,-5.96,-7.53,-4.90


In [50]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create a line plot for each country
fig = make_subplots(specs=[[{"secondary_y": False}]])

# Add a line for each country's export percentage difference
for col in percent_change.columns[1:]:  # Skip the "Quarter" column
    fig.add_trace(
        go.Scatter(x=percent_change["Quarter"], y=percent_change[col], mode='lines', name=col)
    )

# Update layout
fig.update_layout(
    title="Percentage Difference in Export Values per Country",
    xaxis_title="Year, Quarter",
    yaxis_title="Percentage Difference (%)",
    template="plotly_white"
)

# Show the plot
fig.show()